<h1 id="tocheading">LeetCode_Problems_II</h1>
<div id="toc"></div>

In [39]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Find Minimum Height Tree
given a tree, we are asked to find the vertices such that using them as roots, the resulting tree has minimum heights
mathematically, this is equivalent to find middle points in the longest path in the tree, and it is a single node if the
longest path has even total length and two nodes otherwise.

## Algorithm
My algorithm starts from a random (set to be zero) node as root and traverse the tree in a DFS way, each vertex has a   
depth which computes the height of the subtree away from the root. Then we search the neighboring points of root. 
If there is a single subtree whose depth is at least two more than all the other subtree, we move to the child root
Else, either the deepest subtree is of the same height as second deepest subtree, in which case the current root is unique
root of a MHT; or they differ by 1 in depth in which case both current root and the deepest subtree child root are returned

dfs() is implemented to perform the traversal and calculate depth, it's done in $\mathcal{O}(n)$-time
 
checkMHT() checks if we have found the solution, the neighborhood depth are sorted and compared, when the degrees d remain 
bounded by a constant, this is roughly constant time $\mathcal{O}(d)$

move() changes the node under consideration and update the depth of the current node, it's computed in $\mathcal{O}(d)$ time

In [ ]:
class Solution(object):
    def findMinHeightTrees(self, n, edges):
        """
        :type n: int
        :type edges: List[List[int]]
        :rtype: List[int]
        """
        if n < 1:
            return []
        if n == 1:
            return [0]
        if n == 2:
            return [0, 1]

        visited = [False for _ in range(n)]
        neighbors = [[] for _ in range(n)] # indexes for neighborhood nodes
        depth = [0 for _ in range(n)]
        node = 0
        self.found = False
        
        for edge in edges:
            neighbors[edge[0]].append(edge[1])
            neighbors[edge[1]].append(edge[0])

        self.dfs(visited, node, neighbors, depth)

        while not self.found:
            next_node = self.checkMHT(node, neighbors, depth)
            self.move(node, next_node[0], neighbors, depth)
            node = next_node[0]
        return next_node
    
    def move(self, curr_node, next_node, neighbors, depth):
        """
        move to the node with largest depth from current node, update the depth function
        """
        h = 0
        for v in neighbors[curr_node]:
            if v != next_node:
                h = max(h, 1 + depth[v])
        depth[curr_node] = h
        return
        
    
    def checkMHT(self, node, neighbors, depth):
        """
        check if the current node is a candidate for minimum height tree
        if so, change the flag and return the List[Nodes]
        if not, provide the direction to move List[Node]
        """
        if len(neighbors[node]) == 1: # by our constraint, a leaf is not MHT
            return neighbors[node]
    
        heights = sorted([[depth[v], v] for v in neighbors[node]], key = lambda x : x[0])
        
        if heights[-1][0] == heights[-2][0]: # there are two subtree with same largest depth, unique MHT
            self.found = True
            return [node]
        elif heights[-1][0] == heights[-2][0] + 1:
            self.found = True
            return [node, heights[-1][1]] # both current node and the root of deepest subtree are MHT
        else:
            return [heights[-1][1]] # move to root of deepest subtree
        
    def dfs(self, visited, node, neighbors, depth):
        visited[node] = True
        h = 0 # default height
        for v in neighbors[node]:
            if not visited[v]:
                self.dfs(visited, v, neighbors, depth)
                h = max(h, depth[v] + 1) # update the height
        depth[node] = h
        return 
        

BFS is a much simpler method, we successive remove leafs from the tree until we are left with 1 or 2 nodes

In [ ]:
class Solution(object):
    def findMinHeightTrees(self, n, edges):
        """
        :type n: int
        :type edges: List[List[int]]
        :rtype: List[int]
        """
        if n < 1:
            return []
        if n == 1:
            return [0]
        
        neighbors = [[] for _ in range(n)]
        degree = [0] * n
        
        for edge in edges:
            neighbors[edge[0]].append(edge[1])
            neighbors[edge[1]].append(edge[0])
            degree[edge[0]] += 1
            degree[edge[1]] += 1
        
        print neighbors
        print degree
        
        vertices = range(n)
        unvisited = n
        while unvisited > 2:
            leaf = []
            for v in vertices:
                if degree[v] == 1: # it is a leaf
                    leaf.append(v)
            for v in leaf:
                vertices.remove(v)
                w = neighbors[v][0]
                neighbors[w].remove(v)
                degree[w] -= 1
                unvisited -= 1
            
        return vertices 

In [ ]:
s = Solution()
n = 4
edges = [[1, 0], [1, 2], [1, 3]]
s.findMinHeightTrees(n, edges)

# Word Break
Given a non-empty string s and a dictionary wordDict containing a list of non-empty words, determine if s can be segmented into a space-separated sequence of one or more dictionary words. 

You may assume the dictionary does not contain duplicate words.

## Algorithm
The solution is optimized to be done using dynamic programming. Note that the problem does not require the words in the wordDict 
to be used only once. The solution is based on the following recursive procedure:

The word has to be partitioned into the form s[:cut] and s[cut:] for some cut point (can be 0) and we need the subword s[:cut] to be constructable and s[cut:] to be a word in wordDict. Rather than performing a recursive function, we keep track of whether the subword s[:i] can be constructed. This makes the full algorithm to run in $\mathcal{O}$(len(word) * len(wordDict)) time.

This is desired, for example, if instead we use a recursive procedure, a subword will be computed several times and the running time will go exponential. 

In [ ]:
class Solution(object):
    def wordBreak(self, s, wordDict):
        """
        :type s: str
        :type wordDict: List[str]
        :rtype: bool
        """
        dp = [False] * len(s)
        for i in xrange(len(s)):
            for word in wordDict:
                if (word == s[i-len(word) + 1:i + 1]) and (i - len(word) == -1 or dp[i-len(word)] is True):
                    dp[i] = True
        return dp[-1]

In [ ]:
class Solution(object):
    def wordBreak(self, s, wordDict):
        """
        :type s: str
        :type wordDict: List[str]
        :rtype: bool
        """
        dp = [True] # stores where we can build the partial word s[:i]
        for i in xrange(1, len(s) + 1):
            ok = any(dp[j] and s[j:i] in wordDict for j in xrange(i))
            dp.append(ok)
        return dp[-1]

In [ ]:
s = Solution()
w = "leetcode"
wordDict = ["leet", "code"]
s.wordBreak(w, wordDict)

# Gas Station
There are N gas stations along a circular route, where the amount of gas at station i is gas[i].

You have a car with an unlimited gas tank and it costs cost[i] of gas to travel from station i to its next station (i+1).

You begin the journey with an empty tank at one of the gas stations.

Return the starting gas station's index if you can travel around the circuit once, otherwise return -1.

## Algorithm
The solution is simple, based on the following proposition which can be proved mathematically
Let $a[i] = gas[i] - cost[i]$, then there exist an index $k$ such that:
\begin{align}
    & a[k] \ge 0 \\
    & a[k] + a[k + 1] \ge 0 \\
    & ... \\
    & a[k] + a[k + 1] + ... + a[n-1] + a[0] + ... + a[k - 1] \ge 0
\end{align}
if and only if the last equality $\sum_{i = 0}^{n-1} a[i] \ge 0$, and the index $k$ can be chosen to be the such that the partial sum $\sum_{i=0} ^{k-1} a[i]$ is minimum (most negative).

In [ ]:
class Solution(object):
    def canCompleteCircuit(self, gas, cost):
        """
        :type gas: List[int]
        :type cost: List[int]
        :rtype: int
        """
        n = len(gas)
        if sum(gas) < sum(cost): # this is the only case where we can't finish the game
            return -1
        
        aggregate = [0] * n # if we start from 0, agg[i] is the gas we would have when reach station i + 1
        start = 0
        tough = 0
        
        for i in xrange(n):
            extra = gas[i] - cost[i]
            aggregate[i] = aggregate[i - 1] + extra
            if aggregate[i] < tough:
                start = i + 1 # skip the toughest part
                tough = aggregate[i] # update the toughest part
        
        return start % n        

In [ ]:
s = Solution()
gas = [1, 2, 3, 4, 5]
cost = [4, 3, 1, 5, 2]
s.canCompleteCircuit(gas, cost)

# Integer Replacement
Given a positive integer n and you can do operations as follow:

If n is even, replace n with n/2.

If n is odd, you can replace n with either n + 1 or n - 1.

What is the minimum number of replacements needed for n to become 1?

## Algorithm
Rather simple, we have to divide by 2 when the number is even for otherwise it is not optimal. 

If it is odd, unless the number is 3(tricky corner case), we decide it by mod4.

In [ ]:
class Solution(object):
    def integerReplacement(self, n):
        """
        :type n: int
        :rtype: int
        """
        count = 0
        while n > 1:
            if n == 3:
                n = 1
                count += 2
            elif n % 2 == 0:
                n = n / 2
                count += 1
            elif n % 4 == 1:
                n = (n - 1) / 4
                count += 3
            else:
                n = (n + 1) /4
                count += 3
        return count
            
        

# Jump Game
Given an array of non-negative integers, you are initially positioned at the first index of the array.

Each element in the array represents your maximum jump length at that position.

Determine if you are able to reach the last index.

For example:

A = [2,3,1,1,4], return true.

A = [3,2,1,0,4], return false.

## algorithm
We keep track of the index that can be reached so far.

A = [2, 3, 1, 1, 4],

i = 0

boundary = i + A[i] = 2 < 4 

So we can move to place 1 and 2, we increment i by 1 and look at 

i = 1

jump = i + a[i] = 4 == 4, boundary = max(jump, boundary), we update the largest index that can be reached

So we can move to 4 already, terminate.

In the case of failure, A = [3, 2, 1, 0, 4], when we increment i to i = 3, the boundary is always 3, and we can't continue.

In [ ]:
class Solution(object):
    def canJump(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        n = len(nums)
        boundary = 0 # we can at least reach index 0
        for i in xrange(n):
            """
            perform the computation
            """
            boundary = max(boundary, nums[i] + i)
            if boundary >= n - 1:
                return True
            elif boundary == i:
                return False
            else:
                continue           

In [ ]:
s = Solution()
tests = [[3, 2, 1, 0, 4], [2, 3, 1, 1, 4]]
for test in tests:
    print test, s.canJump(test)

# Three Sums
Given an array S of n integers, find three integers in S such that the sum is closest to a given number, target. 

Return the sum of the three integers. 

You may assume that each input would have exactly one solution.

For example, given array S = {-1 2 1 -4}, and target = 1.

The sum that is closest to the target is 2. (-1 + 2 + 1 = 2).

## Algorithm
The solution is a two pointer algorithm which scans the array after it is sorted. Since the time complexity is $\mathcal{O}(n^2)$, the sorting at the beginning does not increase the asympototic complexity. We can't implement a binary search since we are not guaranteed to have an exact hit but only looking for closet sum.

In [ ]:
class Solution(object):
    def threeSumClosest(self, nums, target):
        """
        :type nums: List[int]
        :type target: int
        :rtype: int
        """
        nums.sort()
        n = len(nums)
        if n <= 3:
            return sum(nums)
        
        # compute the margin   
        diff = self.diff(nums, 0, 1, 2, target)
        margin = abs(diff)
        result = target + diff
        
        for i in xrange(n - 2):
            num = nums[i]
            j = i + 1 
            k = n - 1
            while j < k:
                diff = self.diff(nums, i, j, k, target)
                dist = abs(diff)
                if dist < margin:
                    margin = dist # update distance
                    result = target + diff # update sum
                
                # update the pointers
                if diff < 0:
                    j += 1
                elif diff == 0:
                    return target
                else:
                    k -= 1
        return result
    
    def diff(self, nums, i, j, k, target):
        return nums[i] + nums[j] + nums[k] - target

# Next Permutation
Implement next permutation, which rearranges numbers into the lexicographically next greater permutation of numbers.

If such arrangement is not possible, it must rearrange it as the lowest possible order (ie, sorted in ascending order).

The replacement must be in-place, do not allocate extra memory.

Here are some examples. Inputs are in the left-hand column and its corresponding outputs are in the right-hand column.

In [ ]:
class Solution(object):
    def nextPermutation(self, nums):
        """
        :type nums: List[int]
        :rtype: void Do not return anything, modify nums in-place instead.
        """
        n = len(nums)
        
        if n < 2:
            return
        
        """
        In order for a next greater element to exist, the tail of the sequence must have a shape of:
        nums[left_end] < nums[left_end + 1] >= .....>= nums[end], otherwise, the nums is largest already and
        we skip this step
        """
        left_end = -1
        
        for i in xrange(n - 1, 0, -1):
            if nums[i - 1] < nums[i]:
                left_end = i - 1
                break
        
        if left_end != -1: # we find such a left_end
            lo = left_end + 1    
            hi = n - 1
            target = nums[left_end]
            """
            search for the place to insert nums[left_end] in the descending sequence
            nums[left_end + 1] >= ....>= nums[n - 1], we use the binary sort
            """
            while lo < hi - 1:
                mid = (lo + hi) // 2
                if nums[mid] > target:
                    lo = mid
                else:
                    hi = mid
            
            # do the appropriate swap
            if nums[hi] <= target:
                nums[lo], nums[left_end] = nums[left_end], nums[lo]
            else:
                nums[hi], nums[left_end] = nums[left_end], nums[hi]
        
        
        # finally reverse the tail numbers nums[left_end + 1],..., nums[n - 1]
        for i in xrange(left_end + 1, (n + left_end + 1) // 2):
            nums[i], nums[left_end + n - i] = nums[left_end + n - i], nums[i]
        
        return

# Permutations II
Given a collection of numbers that might contain duplicates, return all possible unique permutations.

For example,
[1,1,2] have the following unique permutations:
[
  [1,1,2],
  [1,2,1],
  [2,1,1]
]

In [ ]:
class Solution(object):
    def permuteUnique(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        ans = [[]]
        for n in nums:
            ans = [l[:i] + [n] + l[i:] 
            for l in ans 
            for i in xrange((l + [n]).index(n) + 1)]
            
        return ans

# Permutation Sequence
The set [1,2,3,…,n] contains a total of n! unique permutations.

By listing and labeling all of the permutations in order,

We get the following sequence (ie, for n = 3):

"123"
"132"
"213"
"231"
"312"
"321"

Given n and k, return the kth permutation sequence.

In [ ]:
class Solution(object):
    def getPermutation(self, n, k):
        """
        :type n: int
        :type k: int
        :rtype: str
        """
        
        from math import factorial as fac 
        num = [str(x) for x in range(1, n+1)] # initialize a list contains 1 to n
        perm = [] # This is the returned value
        while n > 0:
            d, k = (k - 1) / fac(n-1), k % fac(n-1)
            perm.append(num.pop(d)), 
            n -= 1
        return ''.join(perm)

# Binary Tree Level Order Traversal II
Given a binary tree, return the bottom-up level order traversal of its nodes' values. (ie, from left to right, level by level from leaf to root).

For example:
    
Given binary tree [3,9,20,null,null,15,7],
    3
   / \
  9  20
    /  \
   15   7
return its bottom-up level order traversal as:

[
  [15,7],
  [9,20],
  [3]
]

In [ ]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Solution(object):
    def levelOrderBottom(self, root):
        """
        :type root: TreeNode
        :rtype: List[List[int]]
        """
        if not root:
            return []
        
        level = [root]
        lst = []
        lst.append([root.val])
        
        
        while True:
            next_level = []
            level_val = []
            for node in level:
                if node.left:
                    next_level.append(node.left)
                    level_val.append(node.left.val)
                if node.right:
                    next_level.append(node.right)
                    level_val.append(node.right.val)
            if not next_level:
                break
            level = next_level
            lst.append(level_val)
        
        return lst[::-1]                

# Binary Tree Preorder Traversal
Given a binary tree, return the preorder traversal of its nodes' values.

For example:
Given binary tree {1,#,2,3},
   1
    \
     2
    /
   3
return [1,2,3].

## Algorithm
Recursive solution with a helper function to save the traversal records in a single list result.

The second solution is iterative, with a stack! to store the visits. This is a little tricky, we need to visit in the order:

root, left_subtree, right_subtree, this is different from the level-order: root, left_node, right_node, which is not recursive

as long as we are working with a recursive call, a Stack structure is needed.

In [ ]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Solution(object):
    def preorderTraversal(self, root):
        """
        :type root: TreeNode
        :rtype: List[int]
        """
        result = []
        self.helper(result, root)
        return result
        
    def helper(self, result, root):
        if not root:
            return
        result.append(root.val)
        self.helper(result, root.left)
        self.helper(result, root.right)
        return 

In [ ]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Solution(object):
    def preorderTraversal(self, root):
        """
        :type root: TreeNode
        :rtype: List[int]
        """
        if not root:
            return []
        result = []
        to_be_visited = [root]
        
        while to_be_visited:
            node = to_be_visited.pop()
            result.append(node.val)
            if node.right:
                to_be_visited.append(node.right)
            if node.left:
                to_be_visited.append(node.left)
    
        return result

# Binary Watch
A binary watch has 4 LEDs on the top which represent the hours (0-11), and the 6 LEDs on the bottom represent the minutes (0-59).

Each LED represents a zero or one, with the least significant bit on the right. 

For example, the above binary watch reads "3:25".

Given a non-negative integer n which represents the number of LEDs that are currently on, 

return all possible times the watch could represent.

Input: n = 1

Return: ["1:00", "2:00", "4:00", "8:00", "0:01", "0:02", "0:04", "0:08", "0:16", "0:32"]

## Algorithm
My own algorithm builds the dictionary of LED and possible hour/minute it shows. And iteratively try to decompose the num into
LEDs. The list comprehension solution is short and elagant, but requieres more computation each time it is called.

In [25]:
class Solution(object):
    def readBinaryWatch(self, num):
        """
        :type num: int
        :rtype: List[str]
        """
        def count_bits(n):
            """
            a function that count the number of 1's in the 
            binary representation of n
            :type n: int
            :rtype: int
            """
            count = 0
            while n > 0:
                count += n % 2
                n >>= 1
            return count
        
        # generate the hours key-value pairs
        # number of LED on top : List[str(possible hours)]
        hours = {}
        for hour in xrange(12):
            digit = count_bits(hour)
            show = str(hour)
            if digit in hours:
                hours[digit].append(show)
            else:
                hours[digit] = [show]
        
        minutes = {}
        for minute in xrange(60):
            digit = count_bits(minute)
            # change the format of outputs
            if minute < 10:
                show = '0' + str(minute)
            else:
                show = str(minute)
                
            if digit in minutes:
                minutes[digit].append(show)
            else:
                minutes[digit] = [show]
        
        if num < 0 or num > 10: # invalid number of LED 
            return []
        
        result = []
        for dh in xrange(min(num + 1, 4)): # number of LED for hour
            dm = num - dh 
            if dm > 5: # invalid number of LED for minute
                continue
            else:
                for hour in hours[dh]:
                    for minute in minutes[dm]:
                        show = hour + ':' + minute
                        result.append(show)
        
        return result

In [ ]:
"""
Alternative simple method, note the use of count and list comprehension
"""
def readBinaryWatch(self, num):
    return ['%d:%02d' % (h, m)
            for h in range(12) for m in range(60)
            if (bin(h) + bin(m)).count('1') == num]

# Subsets II
Given a collection of integers that might contain duplicates, nums, return all possible subsets.

Note: The solution set must not contain duplicate subsets.

For example, if nums = [1,2,2], a solution is:

[
  [2],
  [1],
  [1,2,2],
  [2,2],
  [1,2],
  [],
]


## Algorithm
The first solution sorts the numbers first. This allows us to keep track of the subsets from those already constructed that can
be inserted with a new number. If the new number has not appeared before, every single subset can be copied and inserted with, 
otherwise, it coincide with a previous value and only those subsets the previous element modified can be furhter copied and
inserted. To see this, we look at the example with nums = [1, 2, 2, 3, 3], 

The last 1 subsets in current result can be modified
The new subsets are [[1]]

The last 2 subsets in current result can be modified
The new subsets are [[1, 2], [2]]

The last 2 subsets in current result can be modified
The new subsets are [[2, 2], [1, 2, 2]]

The last 6 subsets in current result can be modified
The new subsets are [[1, 2, 2, 3], [2, 2, 3], [2, 3], [1, 2, 3], [1, 3], [3]]

The last 6 subsets in current result can be modified
The new subsets are [[3, 3], [1, 3, 3], [1, 2, 3, 3], [2, 3, 3], [2, 2, 3, 3], [1, 2, 2, 3, 3]]

And the final answer is a combination of all of them.
[[],
 [1],
 [1, 2],
 [2],
 [2, 2],
 [1, 2, 2],
 [1, 2, 2, 3],
 [2, 2, 3],
 [2, 3],
 [1, 2, 3],
 [1, 3],
 [3],
 [3, 3],
 [1, 3, 3],
 [1, 2, 3, 3],
 [2, 3, 3],
 [2, 2, 3, 3],
 [1, 2, 2, 3, 3]]


In [35]:
class Solution(object):
    def subsetsWithDup(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        n = len(nums)
        result = [[]]
        if n == 0:
            return result
        
        nums.sort() 
        for i in xrange(n):
            if i == 0 or nums[i-1] != nums[i]:
                to_modify = len(result) # update the number of subsets to insert new element
            new_subset = [result[-j - 1] + [nums[i]] for j in xrange(to_modify)] # be careful with negative index
            print 'The last {:d} subsets in current result can be modified'.format(to_modify)
            print 'The new subsets are', new_subset
            result.extend(new_subset)
        return result
    

# Count Numbers with Unique Digits
Given a non-negative integer n, count all numbers with unique digits, x, where 0 ≤ x < 10n.

Example:
Given n = 2, return 91. 

The answer should be the total numbers in the range of 0 ≤ x < 100, excluding [11,22,33,44,55,66,77,88,99])

In [ ]:
class Solution(object):
    def countNumbersWithUniqueDigits(self, n):
        """
        :type n: int
        :rtype: int
        """
        if n == 0:
            return 1

        counts = [1] * (n + 1) # counts[k] is the number of unique digit numbers of length k + 1 
        counts[1] = 9
        
        for i in xrange(2, n + 1):
            counts[i] = counts[i - 1] * (10 - i + 1)
        
        return sum(counts)

In [38]:
candidates = [1, 5, 4]
candidates = sorted(candidates, reverse = True)
candidates

[5, 4, 1]

# Combination Sum
Given a set of candidate numbers (C) (without duplicates) and a target number (T), find all unique combinations 
in C where the candidate numbers sums to T.

The same repeated number may be chosen from C unlimited number of times.

Note:
All numbers (including target) will be positive integers.
The solution set must not contain duplicate combinations.
For example, given candidate set [2, 3, 6, 7] and target 7, 
A solution set is: 
[
  [7],
  [2, 2, 3]
]

## Algorithm
The solution does a depth-first search. When ever we search, we have the following data:
    1. index: the largest index in candidates that we would have in the solution set
    2. candidates: the set of possible numbers to use # can be set as global variable
    3. solution: the attempt solution so far
    4. target: as it states
    5. results: all the found results so far # can be set as a global variable
Then we check:
    1. target = 0, this is the case of success, update results
    2. target < 0, we overshoot, backtrack
    3. target > 0, keep trying, this is the tricky part, unlike the normal case of a DPS, the next level can be the same index. 
    The code is simply a loop over all indexes. 
We reverse sort the candidates and our search stops early. The submission records improves by 30%.

In [ ]:
class Solution(object):
    def combinationSum(self, candidates, target):
        """
        :type candidates: List[int]
        :type target: int
        :rtype: List[List[int]]
        """
        results = []
        if not candidates:
            return results
        candidates = sorted(candidates, reverse = True) # for early stopping
        solution = []
        
        self.dfs(candidates, target, 0, solution, results)
        return results
    
    def dfs(self, nums, target, index, solution, results):
        """
        with the current solution, current index in candidates, search for the next avaliable solution
        the function modifies results, keep appending solutions
        """
        if target < 0:
            return  # backtracking
        if target == 0:
            results.append(solution[::]) # add a copy of solution to our results
            return 
        for i in xrange(index, len(nums)):
            solution.append(nums[i])
            self.dfs(nums, target-nums[i], i, solution, results)
            solution.pop() # clean up
            
            

# Combination Sum III
Find all possible combinations of k numbers that add up to a number n, given that only numbers from 1 to 9 can be used and each combination should be a unique set of numbers.


Example 1:

Input: k = 3, n = 7

Output: [[1,2,4]]

Example 2:

Input: k = 3, n = 9

Output: [[1,2,6], [1,3,5], [2,3,4]]

## Algorithm
A similar idea as before. 

In a DFS search, we keep adding elements until we have k elements in a stack attemp[].

We check if the sum is equal to the target = n, no matter what happened, we backtrack!!!


In [ ]:
class Solution(object):
    def combinationSum3(self, k, n):
        """
        :type k: int
        :type n: int
        :rtype: List[List[int]]
        """
        if k > 9 or k < 1 or n < sum(range(1, k + 1)) or n > sum(range(9, 9 - k, -1)):
            return []
        
        results = []
        attempt = []
        curr = 0
        
        self.dfs(k, n, attempt, curr, results)
        return results
    
    def dfs(self, k, n, attempt, curr, results):
        if len(attempt) == k:
            if sum(attempt) == n:
                results.append(attempt[::])
            return
        
        for num in xrange(curr + 1, 10):
            attempt.append(num)
            self.dfs(k, n, attempt, num, results)
            attempt.pop() # clean up